In [ ]:
#install scikitplot

!pip install scikit-plot

In [ ]:
#import libraries

from sklearn import preprocessing
import pandas as pd

#import different ML classifiers

from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB
from sklearn.svm import SVC, LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier

#import ML evaluation metrics

from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn import metrics, model_selection

#import scikitplot to plot confusion matrix

import scikitplot as skplt

In [ ]:
#import library for accesssing Google Drive from Google Colab

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#read CSV (dataset) file

gender_prediction = pd.read_csv('/content/drive/My Drive/gender-prediction.csv')
gender_prediction

In [ ]:
# Define features x and target variable y
X = gender_prediction.drop('gender', axis=1)
y = gender_prediction['gender']

# Split the dataset into training and testing sets (2/3 train, 1/3 test)
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=1/3, random_state=42)

# Initialization
log_reg = LogisticRegression()
svm_classifier = SVC()
mlp_classifier = MLPClassifier()

# Fit models
log_reg.fit(X_train, y_train)
svm_classifier.fit(X_train, y_train)
mlp_classifier.fit(X_train, y_train)

# Predict on the test set
log_reg_preds = log_reg.predict(X_test)
svm_preds = svm_classifier.predict(X_test)
mlp_preds = mlp_classifier.predict(X_test)

# Accuracy evaluation
log_reg_accuracy = accuracy_score(y_test, log_reg_preds)
svm_accuracy = accuracy_score(y_test, svm_preds)
mlp_accuracy = accuracy_score(y_test, mlp_preds)

# Calculate instances incorrectly classified
log_reg_incorrect = len(y_test) - accuracy_score(y_test, log_reg_preds, normalize=False)
svm_incorrect = len(y_test) - accuracy_score(y_test, svm_preds, normalize=False)
mlp_incorrect = len(y_test) - accuracy_score(y_test, mlp_preds, normalize=False)

# Print results
print(f"Logistic Regression Incorrectly Classified: {log_reg_incorrect} instances")


**2-Using train/test split ratio of 80/20 incorrectly classified instances becomes 1 from 2 because when i set train/test split ration to 80/20 models have more data for training than testing which leads to better performance**

**3-Most important feature attributes are Beard and hair length**

In [ ]:

# Exclude 'beard' and 'hair_length' attributes
X_without_beard_hair = gender_prediction.drop(['beard', 'hair_length', 'gender'], axis=1)
y_without_beard_hair = gender_prediction['gender']

# One-hot encode categorical variables
X_without_beard_hair_encoded = pd.get_dummies(X_without_beard_hair)

# Split the modified dataset into training and testing sets (80/20 split)
X_train_without_beard_hair, X_test_without_beard_hair, y_train_without_beard_hair, y_test_without_beard_hair = train_test_split(
    X_without_beard_hair_encoded, y_without_beard_hair, test_size=0.2, random_state=42
)

# Initialize Logistic Regression model
log_reg_without_beard_hair = LogisticRegression()

# Fit the model on the modified dataset
log_reg_without_beard_hair.fit(X_train_without_beard_hair, y_train_without_beard_hair)

# Predict on the test set
log_reg_preds_without_beard_hair = log_reg_without_beard_hair.predict(X_test_without_beard_hair)

# Evaluate accuracy
log_reg_accuracy_without_beard_hair = accuracy_score(y_test_without_beard_hair, log_reg_preds_without_beard_hair)

# Calculate instances incorrectly classified
log_reg_incorrect_without_beard_hair = len(y_test_without_beard_hair) - accuracy_score(
    y_test_without_beard_hair, log_reg_preds_without_beard_hair, normalize=False
)

# Print results
print(f"Logistic Regression (without beard and hair_length) Incorrectly Classified: {log_reg_incorrect_without_beard_hair} instances")
print(f"Accuracy without beard and hair_length: {log_reg_accuracy_without_beard_hair}")


**4-By removing two important attributes incorreclty classified instances rised from 2 to 4**

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, LeavePOut
from sklearn.metrics import make_scorer, f1_score
import numpy as np

# Define features X and target variable y
X = gender_prediction.drop('gender', axis=1)
y = gender_prediction['gender']

# Initialization
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)


f1_scorer = make_scorer(f1_score, pos_label='male')

# Monte Carlo Cross-Validation with fewer iterations (e.g., 3)
num_iterations = 3
monte_carlo_f1_scores = cross_val_score(rf_classifier, X_encoded, y, cv=num_iterations, scoring=f1_scorer)

# Leave P-Out Cross-Validation with smaller P (e.g., 2)
leave_p_out = LeavePOut(p=2)
leave_p_out_f1_scores = cross_val_score(rf_classifier, X_encoded, y, cv=leave_p_out, scoring=f1_scorer)

print(f"Monte Carlo Cross-Validation F1 Scores: {monte_carlo_f1_scores}")
print(f"Leave P-Out Cross-Validation F1 Scores: {leave_p_out_f1_scores}")

print(f"Mean F1 Score (Monte Carlo): {np.mean(monte_carlo_f1_scores)}")
print(f"Mean F1 Score (Leave P-Out): {np.mean(leave_p_out_f1_scores)}")
